In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import random
import os
import time
import argparse
import json

In [2]:
class Forest():

    def __init__(self, size: int, probability: int, trials: int):
        self.mc_trials = trials
        self.probability = probability
        self.size = size
        self.current_step = 0
        self.edge_connection = 0
        self.min_path = 0
        self.min_average_path = 0
        self.percolation_p = 0
        self.grid = np.zeros((self.size, self.size), dtype=int)
        self.cluster_mass = np.zeros(self.size*self.size)
        self.cluster_mass_non_zeros = []
        self.average_cluster = 0
        self.max_average_cluster = 0
        self.distribution = np.zeros(self.size*self.size)
        self.monte_carlo_step()

    def next(self, i):
        neighbour = i + 1 if i < self.size - 1 else i
        return neighbour

    def previous(self, i):
        neighbour = i - 1 if i > 0 else i
        return neighbour

    def top_neighbour(self, i, j):
        neighbour = self.grid[i - 1][j] if i > 0 else 0
        return neighbour

    def left_neighbour(self, i, j):
        neighbour = self.grid[i][j - 1] if j > 0 else 0
        return neighbour

    def monte_carlo_step(self):
        for i in range(self.size):
            for j in range(self.size):
                r = random.uniform(0, 100)
                type = 1 if r < self.probability else 0
                self.grid[i][j] = type

    def plot(self, tekst):
        colors = ['#FFFFFF','#000000']
        tmp = matplotlib.colors.ListedColormap(colors)
        plt.imshow(self.grid, cmap=tmp);
        plt.colorbar()
        plt.savefig(tekst + '.png')
        plt.close()


class Burning(Forest):
    def __init__(self, size: int, probability: int, trials: int):
         super().__init__(size, probability, trials)

    def burning_method(self):
            # initial paremeter for the burning method
            t = 2
            self.set_fire(t)

            state = True
            while state:
                is_burned = False
                for i in range(self.size):
                    for j in range(self.size):
                        if self.grid[i][j] == t:
                            if j == self.size - 1:
                                self.min_path = t - 1
                                return True
                            if self.grid[self.next(i)][j] == 1:
                                self.grid[self.next(i)][j] = t + 1
                                is_burned = True
                            if self.grid[i][self.next(j)] == 1:
                                self.grid[i][self.next(j)] = t + 1
                                is_burned = True
                            if self.grid[self.previous(i)][j] == 1:
                                self.grid[self.previous(i)][j] = t + 1
                                is_burned = True
                            if self.grid[i][self.previous(j)] == 1:
                                self.grid[i][self.previous(j)] = t + 1
                                is_burned = True

                if is_burned == False: return False
                t += 1

    def set_fire(self, t: int):
        for i in range(self.size):
            if self.grid[i][0] == 1:
                self.grid[i][0] = t

    def simulation_burning(self):
        self.edge_connection = 0

        while self.current_step < self.mc_trials:
            self.min_path = 0
            self.monte_carlo_step()
            if self.burning_method() == True: self.edge_connection += 1
            self.current_step += 1
            self.min_average_path += self.min_path

        self.min_average_path = self.min_average_path / self.mc_trials
        self.percolation_p = self.edge_connection / self.mc_trials

In [3]:
def main_burning(lattice_size: int, trials: int, p_start: float,
                 p_end: float, dp: float):

    file_name = f"burning_L_{lattice_size}_T_{trials}.dat"
    probability, p_end, dp = p_start * 100, p_end * 100, dp * 100

    with open(file_name, "w") as file_to_save:
        while probability <= p_end:
            forest = Burning(lattice_size, probability, trials)
            forest.simulation_burning()
            pattern = f"{round(probability / 100, 3)} \t {round(forest.percolation_p, 2)} \t {forest.min_average_path} \n"
            file_to_save.write(pattern)
            print(probability)
            probability += dp

In [6]:
forest = Burning(30, 0.6, 1)
forest.simulation_burning()
forest.plot("test1")

In [4]:
def parameters():
    config = {
      "lattice_size": 30,
      "trials": 1000,
      "p_start": 0,
      "p_end": 1,
      "dp": 0.01
      }
    return config

kwargs = parameters()
main_burning(**kwargs)

0
1.0
2.0
3.0
4.0
5.0
6.0
7.0
8.0
9.0
10.0
11.0
12.0
13.0
14.0
15.0
16.0
17.0
18.0
19.0
20.0
21.0
22.0
23.0
24.0
25.0
26.0
27.0
28.0
29.0
30.0
31.0
32.0
33.0
34.0
35.0
36.0
37.0
38.0
39.0
40.0
41.0
42.0
43.0
44.0
45.0
46.0
47.0
48.0
49.0
50.0
51.0
52.0
53.0
54.0
55.0
56.0
57.0
58.0
59.0
60.0
61.0
62.0
63.0
64.0
65.0
66.0
67.0
68.0
69.0
70.0
71.0
72.0
73.0
74.0
75.0
76.0
77.0
78.0
79.0
80.0
81.0
82.0
83.0
84.0
85.0
86.0
87.0
88.0
89.0
90.0
91.0
92.0
93.0
94.0
95.0
96.0
97.0
98.0
99.0
100.0


In [5]:
def plot_percolation_threshold(trials, lattice_size):
    color_list = ['blue','red','green','yellow','cyan']
    markers_list = ['.','s','^','o','p']
    data = [np.loadtxt(f"/content/burning_L_30_T_1000.dat") for i in lattice_size]
    axes = plt.gca()
    axes.set_xlim([0,1])
    axes.set_ylim([-0.05,1.05])
    for i in range(len(lattice_size)):
        sub_data = data[i]
        plt.scatter(sub_data[:,0],sub_data[:,1],s=20,marker=markers_list[i],facecolors='none',
                        edgecolors=color_list[i],label=f"L={lattice_size[i]},T={trials}")
    plt.legend()
    plt.xlabel('p')
    plt.ylabel('Percolation probability')
    plt.grid(linewidth=0.5)
    plt.savefig('plot_percolation_threshold.png')
    plt.close()

def plot_min_path(trials, lattice_size):
    color_list = ['blue','red','green','yellow','cyan']
    markers_list = ['.','s','^','o','p']
    data = [np.loadtxt(f"/content/burning_L_30_T_1000.dat") for i in lattice_size]
    axes = plt.gca()
    axes.set_xlim([0, 1])
    for i in range(len(lattice_size)):
        sub_data = data[i]
        plt.scatter(sub_data[:,0],sub_data[:,2], s=20, marker=markers_list[i],facecolors='none',
                    edgecolors=color_list[i],label=f"L={lattice_size[i]},T={trials}")
    plt.legend()
    plt.xlabel('p')
    plt.ylabel('average shortest path')
    plt.grid(linewidth=0.5)
    plt.savefig('plot_min_path.png')
    plt.close()

trials = 1000
lattice_size = [10,20]
plot_percolation_threshold(trials,lattice_size)
plot_min_path(trials,lattice_size)
